In [2]:
%load_ext autoreload
%autoreload 2

from owlready2 import *
import owlready2
print(owlready2.VERSION)

import pandas as pd
import glob, os, hashlib
import requests, json
from datetime import datetime

import random

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.40


In [3]:
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

In [20]:
import pandas as pd
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

4041 items listed


category
Data Privacy and Security risks     208
Economic risks                      403
Environmental risks                 412
Infrastructure risks                345
Legal risks                          38
Political risks                     166
Public health risks                1291
Social risks                        763
Technological risks                 415
Name: name, dtype: int64

In [21]:
DFC.category.unique()[0]

'Political risks'

In [27]:
for k in range(len(DFC.category.unique())):
    Name = DFC.category.unique()[k]
    A = list(DFC[DFC["category"] == Name].name)
    random.shuffle(A)
    A = A[:100]

    REQ = {
        "context": "Out the following list, classify these risks as a list of 10 general, overarching families of risks. Name only the families of risks, with '*' bullet points. \n\n",
        "question": "* "+"\n* ".join(A),
        "token": TOK3N,
        "overwrite": False,
        "source": "local-classRiskslv1_T3.5",
        "seed" : ""
    }
    x = requests.post(URL+"ask/", json = REQ)
    answer = json.loads(x.text)
    print('### '+Name)
    print(answer["answer"])

### Political risks
* Compliance and governance risks
* Preparedness and resilience risks
* Data and evidence quality risks
* Communication and perception risks
* Policy and coordination risks
* Global collaboration and resource allocation risks
* Political response and political divide risks
* Socioeconomic and cultural factors risks
* Data usage and transparency risks
* Urban development and governance risks
### Social risks
* Social and economic disparities
* Public health and safety awareness
* Occupational safety and risk
* Housing and living conditions
* Cultural and lifestyle implications
* Physical activity and mobility
* Urban planning and infrastructure
* Access to resources and services
* Mental health and well-being
* Compliance and enforcement of regulations
### Infrastructure risks
* Infection transmission risks
* Ventilation system performance and design
* Occupational and residential space planning
* Building resilience and operability
* Supply chain and essential suppl

In [28]:
t = """### Political risks
* Trust and transparency challenges
* Globalization challenges
* Lack of global governance and coordination
* Weak local decision-making mechanisms
* Lack of coordination and consistency in mitigation measures
* Ineffectiveness of interventions
* Bioterrorism threats
* Global coordination challenges
* Lack of transparency and clarity
### Social risks
* Disinformation and Misinformation
* Social and Psychological Effects 
* Social Justice Concerns
* Communication and Messaging
* Access to infrastructure  
* Physical activity and mobility
* Misinformation and Media Risks
* Cultural and Lifestyle Risks
* Vulnerability risks
* Occupational and Employment Risks
* Mental and Emotional Wellbeing
* Health Behaviors and Compliance
### Infrastructure risks
* Inadequate remote work setup for employees
* Improper architectural designs
* Inadequate mobility schemes
* Poor urban planning
* Infrastructure maintenance risk 
* Lack of emphasis on air quality 
* Lack of emphasis on water quality 
* Insufficient adoption of health-focused building standards
* Insufficient adoption of health-focused urban standards
* Inadequate hospital design 
* Inadequate adaptability to emerging problems  
* Disruptions from outbreaks and other major external events
### Public health risks
* Failure to implement preventive measures
* Misinterpretation of data and results
* Lack of access to health services
* Public health messaging and education
* Lack of effective medical solutions
* Healthcare resource shortage and management
* Workplace safety and occupational exposure
* Poor access to Diagnostics and Treatment
* Poor vector-borne diseases control
### Data Privacy and Security risks
* Data Security risks
* Privacy risks
* Cybersecurity Threats
* Inaccurate or Incomplete Data
* Misuse of Personal Data
* Reliability and Quality of Data
* Surveillance and Monitoring Privacy Concerns
* Limited Accessibility and Data Equity
* Transparency and Accountability Issues
* Potential Misinformation and False Data
### Economic risks
* Supply chain disruptions
* Pandemic impact on markets
* Lower investment
* Lower employment
* Discontinuity of services
* Global recession and economic instability
* Construction industry impacts
* Resource and capability risks
* Workforce and labor shortages
* Financial strains and liquidity crises
* Food supply chain disruptions 
### Legal risks
* Compliance and Contractual
* Data and Research Integrity
* Intellectual Property
* Workplace Safety and Health
* Privacy and Civil Liberties
* Fraud and Misconduct
* Healthcare Legislation
* Building Codes and Certification
* Technology Deployment and Surveillance
### Environmental risks
* Climate change
* Pollution increase
* Environmental health risks
* Air quality and pollution effects
* Urbanization and habitat loss
* Climate change effects
* Wastewater and sanitation risks
* Biodiversity and ecological impacts
* Energy consumption and sustainability issues
### Technological risks
* Data privacy and security risks
* Healthcare and medical risks
* Infrastructure and connectivity risks
* Modeling and simulation risks
* Modeling and prediction risks
* Artificial intelligence and machine learning risks
* Inadequate ventilation and air circulation
* Operational and implementation risks 
* Innovation and adoption risks
* Healthcare infrastructure and technology adoption risks """

In [29]:
DICT = {}
for l in t.split("\n"):
    if l.startswith("###"):
        DICT[l.replace("### ","")] = []
        na = l.replace("### ","")
    if l.startswith("*"):
        DICT[na].append(l.replace("* ",""))

In [30]:
DICT

{'Political risks': ['Trust and transparency challenges',
  'Globalization challenges',
  'Lack of global governance and coordination',
  'Weak local decision-making mechanisms',
  'Lack of coordination and consistency in mitigation measures',
  'Ineffectiveness of interventions',
  'Bioterrorism threats',
  'Global coordination challenges',
  'Lack of transparency and clarity'],
 'Social risks': ['Disinformation and Misinformation',
  'Social and Psychological Effects ',
  'Social Justice Concerns',
  'Communication and Messaging',
  'Access to infrastructure  ',
  'Physical activity and mobility',
  'Misinformation and Media Risks',
  'Cultural and Lifestyle Risks',
  'Vulnerability risks',
  'Occupational and Employment Risks',
  'Mental and Emotional Wellbeing',
  'Health Behaviors and Compliance'],
 'Infrastructure risks': ['Inadequate remote work setup for employees',
  'Improper architectural designs',
  'Inadequate mobility schemes',
  'Poor urban planning',
  'Infrastructure m

In [3]:
onto = get_ontology("./WIP.owl").load()

In [4]:
RISKS = [x.label[0] for x in onto.Risk.instances() if len(x.label) >= 1]
len(RISKS)

20897

# Classification

### Doing PESTLE, lv1

In [5]:
CategoriesOfRisks = ['Social risks','Public health risks', 'Technological risks',
       'Infrastructure risks', 'Economic risks',
       'Data Privacy and Security risks', 'Environmental risks',
       'Political risks', 'Legal risks']

In [6]:
def classify_risks_PESTLE(CategoriesOfRisks):
    return [
        {
        "name": "classify_risks_PESTLE",
        "description": "Function used to classify risks from a high level perspective, based on what they  impact the most . ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_risks": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of risk ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The risk being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfRisks,
                                "description": 'The category of risk that matches the risk category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct risk category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_risks"],
        },
    }
]

In [7]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newrisks_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    random.shuffle(RISKS)
    newRisks = [x for x in RISKS if x not in alreadythere]
else:
    newRisks = RISKS
    
if len(newRisks):
    newRisks = [str(x) for x in newRisks]
    NB = len(newRisks)//NChunks
    CATS_FWK = [GOOD]
    for k in range(NB):
        try:
            QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="risks_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the risks from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_risks_PESTLE(CategoriesOfRisks),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalrisksT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"])
            d = d [d["category"].isin(CategoriesOfRisks)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC.to_parquet("../../support/newrisks_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All risks covered")

error
error
error


In [ ]:
import pandas as pd
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

3981


category
Data Privacy and Security risks     204
Economic risks                      397
Environmental risks                 404
Infrastructure risks                344
Legal risks                          36
Political risks                     161
Public health risks                1269
Social risks                        758
Technological risks                 408
Name: name, dtype: int64

## Doing lv2

In [49]:
A = list(DFC[DFC["category"] == "Technological risks"].name)
random.shuffle(A)
A = A[:70]

REQ = {
    "context": "Out the following list, classify these risks as a list of 10 general, overarching families of risk. Name only the families of risks, with '*' bullet points. \n\n",
    "question": "* "+"\n* ".join(A),
    "token": TOK3N,
    "overwrite": overwrite,
    "source": "local-classRiskslv1_T3.5",
    "seed" : ""
}
x = requests.post(URL+"ask/", json = REQ)
answer = json.loads(x.text)
print(answer["answer"])

# Alignment to city indicators

In [ ]:
DICTS_CAO = {}

DICTS_CAO["Structure"] = ["Settlement","Biodiversity","Air","Soil","Water","Communication Network", "Water Cycle", "Energy Cycle","Mobility Network","Nature","Dwelling","Buildings/Blocks","Neighbourhoods/Districts","City/Metropolis","Public Space"," Land Use"]
DICTS_CAO["Interactions"] = ["Living","Working","Shopping","Transport","Health","Education","Arts","Sports","Security","Wealth production", "Wealth Distribution","Commerce/Trade","Finances", "Competitiveness", "Entrepreneurship","Culture/Diversity","Social Expression","Heritage", "Tools&Apps","Open data", "Data in/out", "Performance (equity, resilience, self-sufficiency)"]
DICTS_CAO["Society"] = ["Person", "Family","Visitors","Organizations","Business","Participation","Capacity Development", "Leadership","Visions&Priorities", "Laws&Regulations","Accountability"]


In [176]:
DESC_Step1 = "'Structures' contains Environment (Settlements, Biodiversity, Air, Soil, Water), Infrastructure (communication networks, water, energy, matter cycle, mobility,), and Build domain (Dwelling, housing, land use, public spaces). "
DESC_Step1 += "Inter'actions contains Functions, Economy, Culture and Information, focusing on societal, social, economic and information topics. "
DESC_Step1 += "'Society' contains Citizens and Government (inc Persons, familities, organizations, businesses, leadership, vision, laws and regulation, accountability)."
DESC_Step1 = "Use 'Structures' if the risk is in the physical world, 'Interactions' if it is about 'societal, social, economic and information' aspects and Society if talking about governance. "

In [177]:
classifFwk_step1 = [
        {
        "name": "classify_risks_framework",
        "description": "Function used to classify risks according to the evaluation framework, based on what they  impact the most . ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_risks": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of risk ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The risk being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":["Structure","Interactions","Society"],
                                "description": 'The category of risk that matches the risk category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct risk category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_risks"],
        },
    }
]